In [1]:
import os
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow
from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen



# import tensorflow_models as tfm
# import orbit

2024-05-03 21:44:17.590089: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 21:44:17.590262: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:17.592251: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:17.615501: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-03 21:44:18.027340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder




# tfm.__version__

/home/ghost/anaconda3/envs/uts-dl-tf-trainer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PIL import Image

with Image.open('./dataset/Object_Detection/coco/train/100002.jpg') as img :
    pprint.pprint(img.size)

(1920, 1080)


In [4]:

TRAIN_DATA_DIR='./dataset/Object_Detection/coco/train'
TRAIN_ANNOTATION_FILE_DIR='./dataset/Object_Detection/coco/train/train_annotations.json'
OUTPUT_TFRECORD_TRAIN='./dataset/Object_Detection/coco_tfrecords/train'

# Need to provide
  # 1. image_dir: where images are present
  # 2. object_annotations_file: where annotations are listed in json format
  # 3. output_file_prefix: where to write output convered TFRecords files
!python -m official.vision.data.create_coco_tf_record\
  --logtostderr \
  --image_dir={TRAIN_DATA_DIR} \
  --object_annotations_file={TRAIN_ANNOTATION_FILE_DIR} \
  --output_file_prefix={OUTPUT_TFRECORD_TRAIN} \
  --num_shards=1

2024-05-03 21:44:19.535829: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 21:44:19.535983: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.


2024-05-03 21:44:19.537777: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:19.560088: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-03 21:44:20.004219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-05-03 21:44:20.884547: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 21:44:20.898194: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0503 21:44:20.898348 137910658139968 create_coco_tf_record.py:502] writing to output path: ./dataset/Object_Detection/coco_tfrecords/train


I0503 21:44:21.036466 137910658139968 create_coco_tf_record.py:374] Building bounding box index.
I0503 21:44:21.037474 137910658139968 create_coco_tf_record.py:385] 8 images are missing bboxes.


I0503 21:44:21.563014 137910658139968 tfrecord_lib.py:168] On image 0
I0503 21:44:21.580484 137910658139968 tfrecord_lib.py:168] On image 100
I0503 21:44:21.596745 137910658139968 tfrecord_lib.py:168] On image 200


I0503 21:44:21.612905 137910658139968 tfrecord_lib.py:168] On image 300
I0503 21:44:21.628629 137910658139968 tfrecord_lib.py:168] On image 400
I0503 21:44:21.645094 137910658139968 tfrecord_lib.py:168] On image 500
I0503 21:44:21.661101 137910658139968 tfrecord_lib.py:168] On image 600


I0503 21:44:21.676962 137910658139968 tfrecord_lib.py:168] On image 700
I0503 21:44:21.693543 137910658139968 tfrecord_lib.py:168] On image 800
I0503 21:44:21.709506 137910658139968 tfrecord_lib.py:168] On image 900


I0503 21:44:21.725890 137910658139968 tfrecord_lib.py:168] On image 1000


I0503 21:44:21.816976 137910658139968 tfrecord_lib.py:180] Finished writing, skipped 0 annotations.
I0503 21:44:21.819123 137910658139968 create_coco_tf_record.py:537] Finished writing, skipped 0 annotations.


In [5]:

VALID_DATA_DIR='./dataset/Object_Detection/coco/valid'
VALID_ANNOTATION_FILE_DIR='./dataset/Object_Detection/coco/valid/valid_annotations.json'
OUTPUT_TFRECORD_VALID='./dataset/Object_Detection/coco_tfrecords/valid'

!python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir=$VALID_DATA_DIR \
  --object_annotations_file=$VALID_ANNOTATION_FILE_DIR \
  --output_file_prefix=$OUTPUT_TFRECORD_VALID \
  --num_shards=1

2024-05-03 21:44:22.682306: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 21:44:22.682447: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.


2024-05-03 21:44:22.684283: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:22.706427: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-03 21:44:23.153926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-05-03 21:44:24.034414: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


2024-05-03 21:44:24.048184: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0503 21:44:24.048367 139762368448320 create_coco_tf_record.py:502] writing to output path: ./dataset/Object_Detection/coco_tfrecords/valid
I0503 21:44:24.058980 139762368448320 create_coco_tf_record.py:374] Building bounding box index.
I0503 21:44:24.059188 139762368448320 create_coco_tf_record.py:385] 1 images are missing bboxes.


I0503 21:44:24.220462 139762368448320 tfrecord_lib.py:168] On image 0
I0503 21:44:24.237303 139762368448320 tfrecord_lib.py:168] On image 100


I0503 21:44:24.253665 139762368448320 tfrecord_lib.py:168] On image 200
I0503 21:44:24.287929 139762368448320 tfrecord_lib.py:180] Finished writing, skipped 0 annotations.
I0503 21:44:24.289099 139762368448320 create_coco_tf_record.py:537] Finished writing, skipped 0 annotations.


In [6]:

TEST_DATA_DIR='./dataset/Object_Detection/coco/test'
TEST_ANNOTATION_FILE_DIR='./dataset/Object_Detection/coco/test/test_annotations.json'
OUTPUT_TFRECORD_TEST='./dataset/Object_Detection/coco_tfrecords/test'

!python -m official.vision.data.create_coco_tf_record \
    --logtostderr \
    --image_dir=$TEST_DATA_DIR \
    --object_annotations_file=$TEST_ANNOTATION_FILE_DIR \
    --output_file_prefix=$OUTPUT_TFRECORD_TEST \
    --num_shards=1

2024-05-03 21:44:25.140343: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 21:44:25.140476: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:25.142262: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 21:44:25.164513: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-03 21:44:25.607119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-05-03 21:44:26.487053: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 21:44:26.500806: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0503 21:44:26.500985 132139531827008 create_coco_tf_record.py:502] writing to output path: ./dataset/Object_Detection/coco_tfrecords/test


I0503 21:44:26.511912 132139531827008 create_coco_tf_record.py:374] Building bounding box index.
I0503 21:44:26.512111 132139531827008 create_coco_tf_record.py:385] 0 images are missing bboxes.


I0503 21:44:26.674860 132139531827008 tfrecord_lib.py:168] On image 0
I0503 21:44:26.692599 132139531827008 tfrecord_lib.py:168] On image 100
I0503 21:44:26.709017 132139531827008 tfrecord_lib.py:168] On image 200


I0503 21:44:26.748797 132139531827008 tfrecord_lib.py:180] Finished writing, skipped 0 annotations.
I0503 21:44:26.749412 132139531827008 create_coco_tf_record.py:537] Finished writing, skipped 0 annotations.


In [7]:
train_data_input_path = (
    "./dataset/Object_Detection/coco_tfrecords/train-00000-of-00001.tfrecord"
)
valid_data_input_path = (
    "./dataset/Object_Detection/coco_tfrecords/valid-00000-of-00001.tfrecord"
)
test_data_input_path = (
    "./dataset/Object_Detection/coco_tfrecords/test-00000-of-00001.tfrecord"
)
model_dir = "./trained_model/"
export_dir = "./model/"

exp_config = exp_factory.get_exp_config("fasterrcnn_resnetfpn_coco")

batch_size = 8
num_classes = 5

#
HEIGHT, WIDTH = 256, 256
IMG_SIZE = [HEIGHT, WIDTH, 3]

# Backbone config.
exp_config.task.freeze_backbone = False
exp_config.task.annotation_file = ""

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = num_classes + 1
# exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = (
#     exp_config.task.model.num_classes
# )

# Training data config.
exp_config.task.train_data.input_path = train_data_input_path
exp_config.task.train_data.dtype = "float32"
exp_config.task.train_data.global_batch_size = batch_size
exp_config.task.train_data.parser.aug_scale_max = 1.0
exp_config.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
exp_config.task.validation_data.input_path = valid_data_input_path
exp_config.task.validation_data.dtype = "float32"
exp_config.task.validation_data.global_batch_size = batch_size


device = 'CPU'

train_steps = 1000
exp_config.trainer.steps_per_loop = (
    100  # steps_per_loop = num_of_training_examples // train_batch_size
)

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps = (
    100  # validation_steps = num_of_validation_examples // eval_batch_size
)
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
# exp_config.trainer.optimizer_config.learning_rate.type = "cosine"
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

In [8]:
pprint.pprint(exp_config)

ExperimentConfig(task=MaskRCNNTask(init_checkpoint='gs://cloud-tpu-checkpoints/vision-2.0/resnet50_imagenet/ckpt-28080',
                                   model=MaskRCNN(num_classes=6,
                                                  input_size=[256, 256, 3],
                                                  min_level=2,
                                                  max_level=6,
                                                  anchor=Anchor(num_scales=1,
                                                                aspect_ratios=[0.5,
                                                                               1.0,
                                                                               2.0],
                                                                anchor_size=8.0),
                                                  include_mask=False,
                                                  outer_boxes_scale=1.0,
                                                  backbo

In [9]:
logical_device_names = [
    logical_device.name for logical_device in tf.config.list_logical_devices()
]

if "GPU" in "".join(logical_device_names):
    print("This may be broken in Colab.")
    device = "GPU"
elif "TPU" in "".join(logical_device_names):
    print("This may be broken in Colab.")
    device = "TPU"
else:
    print("Running on CPU is slow, so only train for a few steps.")
    device = "CPU"

if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

if "GPU" in "".join(logical_device_names):
    distribution_strategy = tf.distribute.MirroredStrategy()
elif "TPU" in "".join(logical_device_names):
    tf.tpu.experimental.initialize_tpu_system()
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="/device:TPU_SYSTEM:0")
    distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    print("Warning: this will be really slow.")
    distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

Running on CPU is slow, so only train for a few steps.


2024-05-03 21:44:27.381252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 21:44:27.396072: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
with distribution_strategy.scope():
    task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)

In [11]:
for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
    print()
    print(f"images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}")
    print(f"labels.keys: {labels.keys()}")


images.shape: (8, 256, 256, 3)  images.dtype: tf.float32
labels.keys: dict_keys(['anchor_boxes', 'image_info', 'rpn_score_targets', 'rpn_box_targets', 'gt_boxes', 'gt_classes'])


2024-05-03 21:44:31.449747: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
categories =  [
    { "id": 1, "name": "Ready", },
    { "id": 2, "name": "empty_pod", },
    { "id": 3, "name": "germination", },
    { "id": 4, "name": "pod", },
    { "id": 5, "name": "young", }
]

category_index = dict()

for item in categories:
    category_index[ item['id'] ] = item

tf_ex_decoder = TfExampleDecoder()

In [13]:
def show_batch(raw_records, num_of_examples):
    plt.figure(figsize=(20, 20))
    use_normalized_coordinates = True
    min_score_thresh = 0.30
    for i, serialized_example in enumerate(raw_records):
        plt.subplot(1, 3, i + 1)
        decoded_tensors = tf_ex_decoder.decode(serialized_example)
        image = decoded_tensors["image"].numpy().astype("uint8")
        scores = np.ones(shape=(len(decoded_tensors["groundtruth_boxes"])))
        visualization_utils.visualize_boxes_and_labels_on_image_array(
            image,
            decoded_tensors["groundtruth_boxes"].numpy(),
            decoded_tensors["groundtruth_classes"].numpy().astype("int"),
            scores,
            category_index=category_index,
            use_normalized_coordinates=use_normalized_coordinates,
            max_boxes_to_draw=200,
            min_score_thresh=min_score_thresh,
            agnostic_mode=False,
            instance_masks=None,
            line_thickness=4,
        )

        plt.imshow(image)
        plt.axis("off")
        plt.title(f"Image-{i+1}")
    plt.show()

In [14]:
buffer_size = 20
num_of_examples = 3

raw_records = (
    tf.data.TFRecordDataset(exp_config.task.train_data.input_path)
    .shuffle(buffer_size=buffer_size)
    .take(num_of_examples)
)
show_batch(raw_records, num_of_examples)

2024-05-03 21:44:31.698404: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode="train_and_eval",
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True,
)

restoring or initializing model...


2024-05-03 21:44:48.210724: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


INFO:tensorflow:Customized initialization is done through the passed `init_fn`.


INFO:tensorflow:Customized initialization is done through the passed `init_fn`.


train | step:      0 | training until step 100...


...

2024-05-04 00:05:07.794308: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


2024-05-04 00:05:08.708275: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


creating index...
index created!


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


DONE (t=0.62s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.724
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.285
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.427
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.096
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.491
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.569
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.283
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

In [16]:

%load_ext tensorboard
%tensorboard --logdir './trained_model/'

In [17]:
export_saved_model_lib.export_inference_graph(
    input_type="image_tensor",
    batch_size=1,
    input_image_size=[HEIGHT, WIDTH],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(model_dir),
    export_dir=export_dir,
)

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


In [18]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    image = None
    if path.startswith("http"):
        response = urlopen(path)
        image_data = response.read()
        image_data = BytesIO(image_data)
        image = Image.open(image_data)
    else:
        image_data = tf.io.gfile.GFile(path, "rb").read()
        image = Image.open(BytesIO(image_data))

    (im_width, im_height) = image.size
    return (
        np.array(image.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)
    )


def build_inputs_for_object_detection(image, input_image_size):
    """Builds Object Detection model inputs for serving."""
    image, _ = resize_and_crop_image(
        image,
        input_image_size,
        padded_size=input_image_size,
        aug_scale_min=1.0,
        aug_scale_max=1.0,
    )
    return image

In [19]:
num_of_examples = 3

test_ds = tf.data.TFRecordDataset(
    # test path
    # "./bccd_coco_tfrecords/test-00000-of-00001.tfrecord"
    test_data_input_path
).take(num_of_examples)
show_batch(test_ds, num_of_examples)

2024-05-04 00:05:44.025847: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
imported = tf.saved_model.load(export_dir)
model_fn = imported.signatures["serving_default"]

In [21]:
input_image_size = (HEIGHT, WIDTH)
plt.figure(figsize=(20, 20))
min_score_thresh = (
    0.30  # Change minimum score for threshold to see all bounding boxes confidences.
)

for i, serialized_example in enumerate(test_ds):
    plt.subplot(1, 3, i + 1)
    decoded_tensors = tf_ex_decoder.decode(serialized_example)
    image = build_inputs_for_object_detection(
        decoded_tensors["image"], input_image_size
    )
    image = tf.expand_dims(image, axis=0)
    image = tf.cast(image, dtype=tf.uint8)
    image_np = image[0].numpy()
    result = model_fn(image)
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image_np,
        result["detection_boxes"][0].numpy(),
        result["detection_classes"][0].numpy().astype(int),
        result["detection_scores"][0].numpy(),
        category_index=category_index,
        use_normalized_coordinates=False,
        max_boxes_to_draw=200,
        min_score_thresh=min_score_thresh,
        agnostic_mode=False,
        instance_masks=None,
        line_thickness=4,
    )
    plt.imshow(image_np)
    plt.axis("off")

plt.show()

2024-05-04 00:05:52.172292: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
